In [1]:
from __future__ import print_function
import torch
from torchvision import datasets, transforms
import numpy as np
from WeakLearners import WongNeuralNetCIFAR10
import matplotlib.pyplot as plt

In [2]:
!pip install pytorch_memlab

You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
from Boosting import Ensemble, runBoosting
from AdversarialAttacks import attack_fgsm, attack_pgd
from pytorch_memlab import LineProfiler

In [4]:
import utils

In [5]:
cuda = torch.device('cuda:0')

In [6]:
%load_ext autoreload
%autoreload 2

## CIFAR10 Boosting (Adversarial)

In [7]:
# maxSamples_vals = [100000, 200000, 500000]
# maxSamples_vals = [500000]
maxSamples_vals = [500000, 1000000, 2000000]
# maxSamples_vals = [123456]
# already done for 10000, 50K, 100K
batch_size=100
# maxSamples_vals = [batch_size + 1]

In [8]:
num_wl = 20 # maybe around 100? #later: maybe change this to an array?
num_epochs = 1

In [9]:
ensembles = []
epsilons = [0.0, 0.01, 0.02, 0.03, 0.05, 0.1]
train_eps_nn = 8
# epsilons = []
# epsilons = [0.0, 0.01]

In [10]:
# from Boosting import SchapireWongMulticlassBoosting
# from AdversarialAttacks import attack_fgsm
# # Ensemble.record_accuracies, Ensemble.calc_accuracies, attack_fgsm, Ensemble.schapireContinuousPredict
# with LineProfiler(runBoosting, SchapireWongMulticlassBoosting, WongNeuralNetCIFAR10.fit, WongNeuralNetCIFAR10.batchUpdate) as prof:
#     for i in range(len(maxSamples_vals)):
#         maxSamples = maxSamples_vals[i]
#         ensemble = runBoosting(num_wl, maxSamples, dataset=datasets.CIFAR10, weakLearnerType = WongNeuralNetCIFAR10, val_attacks = [attack_fgsm], 
#                                    attack_eps_nn=epsilons, attack_eps_ensemble=epsilons, train_eps_nn=8, adv_train=True, batch_size=batch_size)
#         ensembles.append(ensemble)
#         path_head = 'results/plots/cifar10/'
#         acc_file = path_head + f'acc_maxSamples_{maxSamples}.png'
#         adv_acc_file = path_head + f'adv_acc_maxSamples_{maxSamples}.png'
#         loss_file = path_head + f'loss_maxSamples_{maxSamples}.png'
#         wl_train_acc_file = path_head + f'wl_train_acc_maxSamples_{maxSamples}.png'
#         ensemble.plot_accuracies(acc_file)
#         ensemble.plot_loss(loss_file)
#         ensemble.plot_adversarial_accuracies(adv_acc_file)
#         ensemble.plot_wl_train_acc(wl_train_acc_file)
# prof.display()

In [ ]:
# from Boosting import SchapireWongMulticlassBoosting
from AdversarialAttacks import attack_fgsm
import gc
# Ensemble.record_accuracies, Ensemble.calc_accuracies, attack_fgsm, Ensemble.schapireContinuousPredict
for i in range(len(maxSamples_vals)):
    print("len(maxSamples_vals):", len(maxSamples_vals))
    maxSamples = maxSamples_vals[i]
    print("maxSamples:", maxSamples)
    ensemble = runBoosting(num_wl, maxSamples, dataset=datasets.CIFAR10, weakLearnerType = WongNeuralNetCIFAR10, val_attacks = [attack_fgsm], 
                               attack_eps_nn=epsilons, attack_eps_ensemble=epsilons, train_eps_nn=train_eps_nn, adv_train=True, batch_size=batch_size)
    print("number of wl in ensemble:", len(ensemble.weakLearners))
    ensembles.append(ensemble)
    gc.collect()
#     for obj in gc.get_objects():
#         try:
#             if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
#                 print(type(obj), obj.size())
#         except:
#             pass
        
    path_head = 'results/plots/cifar10/'
    acc_file = path_head + f'acc_maxSamples_{maxSamples}.png'
    adv_acc_file = path_head + f'adv_acc_maxSamples_{maxSamples}.png'
    loss_file = path_head + f'loss_maxSamples_{maxSamples}.png'
    wl_train_acc_file = path_head + f'wl_train_acc_maxSamples_{maxSamples}.png'
    ensemble.plot_accuracies(acc_file)
    ensemble.plot_loss(loss_file)
    ensemble.plot_adversarial_accuracies(adv_acc_file)
    ensemble.plot_wl_train_acc(wl_train_acc_file)


len(maxSamples_vals): 3
maxSamples: 500000
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
attack eps ens [0.0, 0.01, 0.02, 0.03, 0.05, 0.1]
weakLearners before super call: []
weakLearners after super call: []
cnt:  20
t: 0
cnt2: 1
----------------------------------------------------------------------------------------------------
Training weak learner 0
adv_train: True
Epoch 0


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/cifar.py:118: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  img, target = self.data[index], self.targets[index]


Val accuracy: 0.22
Progress: 10000,  val accuracy: 0.2200
Val accuracy: 0.26
Progress: 20000,  val accuracy: 0.2600
Val accuracy: 0.29
Progress: 30000,  val accuracy: 0.2900
Val accuracy: 0.33
Progress: 40000,  val accuracy: 0.3300
Val accuracy: 0.35
Progress: 50000,  val accuracy: 0.3500


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:893: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


Epoch 1
Val accuracy: 0.35
Progress: 60000,  val accuracy: 0.3500
Val accuracy: 0.36
Progress: 70000,  val accuracy: 0.3600
Val accuracy: 0.37
Progress: 80000,  val accuracy: 0.3700
Val accuracy: 0.4
Progress: 90000,  val accuracy: 0.4000
Val accuracy: 0.37
Progress: 100000,  val accuracy: 0.3700
Epoch 2
Val accuracy: 0.39
Progress: 110000,  val accuracy: 0.3900
Val accuracy: 0.43
Progress: 120000,  val accuracy: 0.4300
Val accuracy: 0.47
Progress: 130000,  val accuracy: 0.4700
Val accuracy: 0.44
Progress: 140000,  val accuracy: 0.4400
Val accuracy: 0.4
Progress: 150000,  val accuracy: 0.4000
Epoch 3
Val accuracy: 0.43
Progress: 160000,  val accuracy: 0.4300
Val accuracy: 0.48
Progress: 170000,  val accuracy: 0.4800
Val accuracy: 0.45
Progress: 180000,  val accuracy: 0.4500
Val accuracy: 0.49
Progress: 190000,  val accuracy: 0.4900
Val accuracy: 0.52
Progress: 200000,  val accuracy: 0.5200
Epoch 4
Val accuracy: 0.55
Progress: 210000,  val accuracy: 0.5500
Val accuracy: 0.51
Progress: 2

In [ ]:
# for maxSamples in maxSamples_vals:
#         ensemble = runBoosting(num_wl, maxSamples, dataset=datasets.CIFAR10, weakLearnerType = WongNeuralNetCIFAR10, val_attacks = [attack_fgsm], 
#                                attack_eps_nn=epsilons, attack_eps_ensemble=epsilons, train_eps_nn=8, adv_train=True)
#         ensembles.append(ensemble)

In [ ]:
ensembles[0].plot_adversarial_accuracies()

In [ ]:
ensembles[0].plot_train_loss()
ensembles[0].plot_val_loss()
ensembles[0].plot_train_accuracies()
ensembles[0].plot_val_accuracies()

In [10]:
ensembles[0].attack_eps

[]